# 🧩 Online Retail – Feature Engineering

This notebook creates the **monthly modeling dataset** used for sales forecasting.

**Objective:** Transform cleaned transactional data into structured features suitable for machine learning.

## 1️⃣ Import libraries and load cleaned data

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [ ]:
# Load both Excel sheets separately
df_2009_2010 = pd.read_excel(
    r"C:\Users\Usuario\Desktop\Nahu\Portfolio-Data\online-retail-forecast-tableau\data\raw\online_retail_II.xlsx",
    sheet_name="Year 2009-2010"
)

df_2010_2011 = pd.read_excel(
    r"C:\Users\Usuario\Desktop\Nahu\Portfolio-Data\online-retail-forecast-tableau\data\raw\online_retail_II.xlsx",
    sheet_name="Year 2010-2011"
)

# Remove overlapping rows from the second sheet
# Excel row 22525 corresponds to index 22524 in pandas (0-based indexing)
df_2010_2011 = df_2010_2011.iloc[22524:].reset_index(drop=True)

# Concatenate both datasets into a single DataFrame
df = pd.concat(
    [df_2009_2010, df_2010_2011],
    ignore_index=True
)

# Convert InvoiceDate column to datetime format
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

# Preview the merged dataset
df.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [3]:
# Basic cleaning (same criteria as EDA)
df = df[(df['Quantity'] > 0) & (df['Price'] > 0)].copy()
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Revenue'] = df['Quantity'] * df['Price']

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


## 2️⃣ Monthly aggregation

In [33]:
df['Month'] = df['InvoiceDate'].dt.to_period('M')

monthly = (
    df.groupby(['Month', 'Country'])
      .agg(
          revenue=('Revenue', 'sum'),
          orders=('Invoice', 'nunique'),
          customers=('Customer ID', 'nunique'),
          quantity=('Quantity', 'sum')
      )
      .reset_index()
)

monthly.head()

,Month,Country,revenue,orders,customers,quantity
0,2009-12,Australia,271.10,2,2,160
1,2009-12,Austria,1998.34,2,2,564
2,2009-12,Belgium,447.60,3,2,153
3,2009-12,Channel Islands,989.18,1,1,439
4,2009-12,Cyprus,3556.98,4,3,1578


Data is aggregated at a **monthly × country** level to reduce noise and align with business reporting practices.

## 3️⃣ Time-based features

In [34]:
#monthly['Month'] = monthly['Month'].dt.to_timestamp('M')
monthly['year'] = monthly['Month'].dt.year
monthly['month'] = monthly['Month'].dt.month
monthly = monthly.sort_values(['Country', 'Month'])

monthly.head()

,Month,Country,revenue,orders,customers,quantity,year,month
0,2009-12,Australia,271.10,2,2,160,2009,12
41,2010-02,Australia,1029.66,3,3,624,2010,2
60,2010-03,Australia,429.39,2,2,191,2010,3
83,2010-04,Australia,630.95,2,2,560,2010,4
104,2010-05,Australia,2371.15,6,4,667,2010,5


## 4️⃣ Lag features

In [35]:
for lag in [1, 2, 3]:
    monthly[f'revenue_lag_{lag}'] = (
        monthly.groupby('Country')['revenue'].shift(lag)
    )

monthly.head()

,Month,Country,revenue,orders,customers,quantity,year,month,revenue_lag_1,revenue_lag_2,revenue_lag_3
0,2009-12,Australia,271.10,2,2,160,2009,12,NaN,NaN,NaN
41,2010-02,Australia,1029.66,3,3,624,2010,2,271.10,NaN,NaN
60,2010-03,Australia,429.39,2,2,191,2010,3,1029.66,271.10,NaN
83,2010-04,Australia,630.95,2,2,560,2010,4,429.39,1029.66,271.10
104,2010-05,Australia,2371.15,6,4,667,2010,5,630.95,429.39,1029.66


Lagged revenue features capture temporal dependency and short-term trends.

## 5️⃣ Rolling statistics

In [27]:
monthly['revenue_roll_3'] = (
    monthly.groupby('Country')['revenue']
           .shift(1)
           .rolling(window=3)
           .mean()
)

monthly['revenue_roll_6'] = (
    monthly.groupby('Country')['revenue']
           .shift(1)
           .rolling(window=6)
           .mean()
)

monthly.head()

,Month,Country,revenue,orders,customers,quantity,year,month,revenue_lag_1,revenue_lag_2,revenue_lag_3,revenue_roll_3,revenue_roll_6
0,2009-12-31,Australia,271.10,2,2,160,2009,12,NaN,NaN,NaN,NaN,NaN
41,2010-02-28,Australia,1029.66,3,3,624,2010,2,271.10,NaN,NaN,NaN,NaN
60,2010-03-31,Australia,429.39,2,2,191,2010,3,1029.66,271.10,NaN,NaN,NaN
83,2010-04-30,Australia,630.95,2,2,560,2010,4,429.39,1029.66,271.10,576.716667,NaN
104,2010-05-31,Australia,2371.15,6,4,667,2010,5,630.95,429.39,1029.66,696.666667,NaN


Rolling averages smooth volatility and approximate seasonality effects.

## 6️⃣ Final dataset for modeling

In [ ]:
model_df = monthly.dropna().copy()
model_df.head(400)

model0_df = monthly.copy()
model0_df.head(9)

,Month,Country,revenue,orders,customers,quantity,year,month,revenue_lag_1,revenue_lag_2,revenue_lag_3
0,2009-12,Australia,271.10,2,2,160,2009,12,NaN,NaN,NaN
41,2010-02,Australia,1029.66,3,3,624,2010,2,271.10,NaN,NaN
60,2010-03,Australia,429.39,2,2,191,2010,3,1029.66,271.10,NaN
83,2010-04,Australia,630.95,2,2,560,2010,4,429.39,1029.66,271.10
104,2010-05,Australia,2371.15,6,4,667,2010,5,630.95,429.39,1029.66
...,...,...,...,...,...,...,...,...,...,...,...
309,2011-01,Poland,613.22,1,1,288,2011,1,248.16,821.14,349.47
332,2011-02,Poland,604.05,2,2,297,2011,2,613.22,248.16,821.14
357,2011-03,Poland,317.78,1,1,162,2011,3,604.05,613.22,248.16
380,2011-04,Poland,705.84,1,1,432,2011,4,317.78,604.05,613.22


In model_df rows with missing lag or rolling values are dropped to ensure model consistency.
In model0_df, no rows are dropped.

## 7️⃣ Save processed dataset

In [56]:
model_df.to_csv(r'../data/processed/monthly_revenue.csv', index=False)
model0_df.to_csv(r'../data/processed/monthly0_revenue.csv', index=False)


The resulting dataset is ready for time-based train/test splitting and machine learning modeling.